In [ ]:
import os
from du_astro_utils import calibration, photometry, utils
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from scipy.ndimage import median_filter
from tqdm import tqdm

In [ ]:
data_dir = os.path.join(utils.C2PU_RES_DIR, utils.DIR_PHOTOM, utils.DIR_GALCLUST)

In [ ]:
reduced = True
aligned = False
for ix, ddir in enumerate(os.listdir(data_dir)):
    subdata_dir = os.path.join(data_dir, ddir)
    if reduced:
        subdata_dir = os.path.join(subdata_dir, "REDUCED")
    if aligned:
        subdata_dir = os.path.join(subdata_dir, "aligned")
    if os.path.isdir(subdata_dir):
        list_fits = [im for im in sorted(os.listdir(subdata_dir)) if "REDUCED.fits" in im]
        list_fits = sorted(list_fits)
        print(f"{ix} - {subdata_dir} : {len(list_fits)} files")
    else:
        print(f"{subdata_dir} : not a directory.")

In [ ]:
if True:
    combtype = "MEDIAN"  # 'AVERAGE', 'SUM'
    center_type = "MOST"
    for filt in tqdm(["g", "r", "i"]):  # tqdm(['g', 'r', 'i', 'z']):
        exp = 60 if filt == "z" else 30 if filt == "i" else 15  # 10
        os.listdir(data_dir)[9] if filt == "z" else os.listdir(data_dir)[8]
        with fits.open(f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}_{center_type}.fits") as stack_ip:
            hdr = stack_ip[0].header
            data = stack_ip[0].data
        mean, med, sigma = sigma_clipped_stats(data, sigma=3)
        plt.imshow(data, cmap="gray", vmin=med, vmax=med + 12 * sigma)
        # plt.colorbar()
        plt.title(f"SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}")
        plt.show()

## References in PANSTARRS

In [ ]:
from du_astro_utils import query_panstarrs

In [ ]:
manual_stack = True
combtype = "MEDIAN"  # SUM ou AVERAGE ou MEDIAN
center_type = "MOST"
filt = "g"  # , 'r', 'i', 'z']:
exp = 60 if filt == "z" else 30 if filt == "i" else 15  # 10
os.listdir(data_dir)[9] if filt == "z" else os.listdir(data_dir)[8]
refname = f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_np{combtype}.fits" if manual_stack else f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}_{center_type}.fits"
ref_ps = query_panstarrs(refname)

In [ ]:
ref_ps

In [ ]:
from astropy.wcs import WCS
from astropy.wcs.utils import skycoord_to_pixel
from astropy.coordinates import Angle, SkyCoord

coord_panstarrs = SkyCoord(ref_ps["RAJ2000"], ref_ps["DEJ2000"])

## Photometry

In [ ]:
manual_stack = False
combtype = "Sum"  # SUM ou AVERAGE ou MEDIAN
center_type = "MOST"

### G-band

In [ ]:
from astropy.table import Table, vstack

from astropy.time import Time
import warnings
from astropy.utils.exceptions import AstropyWarning
from astropy.coordinates.name_resolve import NameResolveError

use_sextractor = True

filt = "g"  # , 'r', 'i', 'z']:
exp = 60 if filt == "z" else 30 if filt == "i" else 15  # 10
os.listdir(data_dir)[9] if filt == "z" else os.listdir(data_dir)[8]
red_sci_image = f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_np{combtype}.fits" if manual_stack else f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}_{center_type}.fits"
ref_ps = query_panstarrs(red_sci_image)
coord_panstarrs = SkyCoord(ref_ps["RAJ2000"], ref_ps["DEJ2000"])
im_dir = os.path.abspath(os.path.dirname(red_sci_image))
im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))
with fits.open(red_sci_image) as hdul:
    hdu = hdul[0]
    wcs = WCS(hdu.header)
    epoch = Time(hdu.header.get("MJD-OBS"), format="mjd", scale="utc")
    if use_sextractor:
        sex_cmd = f"source-extractor -c default.sex {red_sci_image} -CATALOG_NAME {im_name}.cat -CATALOG_TYPE FITS_1.0 -VERBOSE_TYPE QUIET -FILTER_NAME gauss_5.0_9x9.conv"
        os.system(sex_cmd)
        cat_tab_g = Table.read(f"{im_name}.cat")
        cat_tab_g.rename_column("X_IMAGE", "xcentroid")
        cat_tab_g.rename_column("Y_IMAGE", "ycentroid")
        _, fwhm, _ = sigma_clipped_stats(cat_tab_g["FWHM_IMAGE"], sigma=3)
        source_coords_g = SkyCoord(ra=cat_tab_g["ALPHA_J2000"], dec=cat_tab_g["DELTA_J2000"], unit="deg", obstime=epoch)
    else:
        sources_g = photometry.detect_sources(red_sci_image, detection_fwhm=10, verbose=False)
        try:
            fwhm = photometry.get_fwhm(red_sci_image, sources)
        except RuntimeError:
            fwhm = 10
        cat_tab_g = photometry.apert_photometry(red_sci_image, sources_g, fwhm)
        source_coords_g = SkyCoord.from_pixel(cat_tab_g["xcentroid"], cat_tab_g["xcentroid"], wcs)

In [ ]:
cat_tab_g

In [ ]:
plt.scatter(cat_tab_g["CLASS_STAR"], cat_tab_g["ELONGATION"])
plt.ylim(0.0, 5.0)

In [ ]:
plt.hist(cat_tab_g["CLASS_STAR"], bins=50)

In [ ]:
cat_stars_g = cat_tab_g[cat_tab_g["CLASS_STAR"] > 0.7]
star_coords_g = source_coords_g[cat_tab_g["CLASS_STAR"] > 0.7]

In [ ]:
cat_stars_g

In [ ]:
import astropy.units as u

xm_id, xm_ang_distance, _ = star_coords_g.match_to_catalog_sky(coord_panstarrs, nthneighbor=1)
# print(hdu.header.get('PIXSCALX') * fwhm)
# max_sep = hdu.header.get('PIXSCALX') * fwhm * u.arcsec
max_sep = 2.5 * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches_g = star_coords_g[sep_constraint]
catalog_matches_g = ref_ps[xm_id[sep_constraint]]
coord_catalog_matches_g = coord_panstarrs[xm_id[sep_constraint]]

In [ ]:
# Compute instrumental magnitude
if not use_sextractor:
    exptime = hdu.header.get("EXPTIME")
    ins_mag_g = -2.5 * np.log10(cat_stars_g[sep_constraint]["aper_sum_bkgsub"] / exptime)
    cat_mag_g = ref_ps["gmag"][xm_id[sep_constraint]]

    ins_err_g = ins_mag_g - -2.5 * np.log10((cat_stars_g[sep_constraint]["aper_sum_bkgsub"] + cat_stars_g[sep_constraint]["noise"]) / exptime)
    cat_err_g = ref_ps["e_gmag"][xm_id[sep_constraint]]

    cat_stars_g["ins_mag"] = 99
    cat_stars_g["ins_mag"][sep_constraint] = ins_mag_g
else:
    cat_mag_g = ref_ps["gmag"][xm_id[sep_constraint]]
    cat_err_g = ref_ps["e_gmag"][xm_id[sep_constraint]]
    ins_mag_g = cat_stars_g[sep_constraint]["MAG_BEST"]
    ins_err_g = cat_stars_g[sep_constraint]["MAGERR_BEST"]

In [ ]:
sel = ins_mag_g < 99
plt.scatter(ins_mag_g[sel], cat_mag_g[sel])

In [ ]:
plt.scatter(cat_mag_g[sel], ins_mag_g[sel] - cat_mag_g[sel])

In [ ]:
from sklearn import linear_model

# Selection from magnitude range
mag_min, mag_max = 14, 18
cond = (cat_mag_g > mag_min) & (cat_mag_g < mag_max) & (~cat_mag_g.mask) & (~np.isnan(ins_mag_g)) & (ins_mag_g < 99)

# Create two mock arrays for linear regression
X = ins_mag_g[cond].reshape(-1, 1)
y = cat_mag_g[cond].reshape(-1, 1)


# Simple linear regression
linear = linear_model.LinearRegression()
linear.fit(X, y)


# sigma clipping pour choisir le threshold
from scipy import stats

MAD = stats.median_abs_deviation(X - y)
_, _, sig = sigma_clipped_stats(X - y)

print(MAD, sig)

# RANSAC linear regressions
ransac = linear_model.RANSACRegressor(residual_threshold=3 * MAD[0])
# ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)

# Results
print("Photometric calibration:")
print(f"  Linear Slope: {linear.coef_[0][0]:.3f}")
print(f"  Linear ZP   : {linear.intercept_[0]:.3f}\n")
print(f"  RANSAC Slope: {ransac.estimator_.coef_[0][0]:.3f}")
print(f"  RANSAC ZP   : {ransac.estimator_.intercept_[0]:.3f}")

In [ ]:
# Plotting regression
# Outliers and Valid points
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Linear regressions (simple and RANSAC)
line_X = np.arange(X.min(), X.max() + 1)[:, np.newaxis]
line_y_simple = linear.predict(line_X)
line_y_ransac = ransac.predict(line_X)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plot data
ax[0].scatter(X[inlier_mask], y[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[0].scatter(X[outlier_mask], y[outlier_mask], color="gray", marker=".", label="Outliers")

# Plot regressions
ax[0].plot(line_X, line_y_simple, color="cornflowerblue", label="Linear regressor")
ax[0].plot(line_X, line_y_ransac, color="navy", label="RANSAC regressor")

# Axes...
ax[0].legend(loc="lower right")
# ax[0].set_ylim([10,18])
ax[0].set_xlabel("Instrument magnitude")
ax[0].set_ylabel("Catalog magnitude")
ax[0].set_aspect("equal")

_, zp_median, zp_sigma = sigma_clipped_stats(y - X, sigma=3)
ax[1].scatter(y[inlier_mask], y[inlier_mask] - X[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[1].scatter(y[outlier_mask], y[outlier_mask] - X[outlier_mask], color="gray", marker=".", label="Outliers")
ax[1].set_xlabel("Catalog magnitude")

ax[1].axhline(zp_median, label="Median")
ax[1].axhline(zp_median + zp_sigma, linestyle="--", label="Standard deviation")
ax[1].axhline(zp_median - zp_sigma, linestyle="--")
print(f"  sigma  ZP   : {zp_sigma:.3f}")

ax[1].set_ylabel("Instrument - Catalog magnitude")
ax[1].legend(loc="best")

In [ ]:
# Compute calibrated mag
cat_tab_g["AB_MAG"] = 99.0

# Positive values
if not use_sextractor:
    positive = np.where(cat_tab_g["aper_sum_bkgsub"] > 0)
    cat_tab_g["AB_MAG"][positive] = ransac.predict((-2.5 * np.log10(cat_tab_g[positive]["aper_sum_bkgsub"] / exptime)).data.reshape(-1, 1)).flatten()
else:
    positive = np.where(cat_tab_g["FLUX_BEST"] > 0)
    cat_tab_g["AB_MAG"][positive] = ransac.predict(cat_tab_g[positive]["MAG_BEST"].data.reshape(-1, 1)).flatten()
cat_tab_g

### R- band

In [ ]:
filt = "r"
exp = 60 if filt == "z" else 30 if filt == "i" else 15  # 10
os.listdir(data_dir)[9] if filt == "z" else os.listdir(data_dir)[8]
red_sci_image = f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_np{combtype}.fits" if manual_stack else f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}_{center_type}.fits"
ref_ps = query_panstarrs(red_sci_image)
coord_panstarrs = SkyCoord(ref_ps["RAJ2000"], ref_ps["DEJ2000"])
im_dir = os.path.abspath(os.path.dirname(red_sci_image))
im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))
with fits.open(red_sci_image) as hdul:
    hdu = hdul[0]
    wcs = WCS(hdu.header)
    epoch = Time(hdu.header.get("MJD-OBS"), format="mjd")
    if use_sextractor:
        sex_cmd = f"source-extractor -c default.sex {red_sci_image} -CATALOG_NAME {im_name}.cat -CATALOG_TYPE FITS_1.0 -VERBOSE_TYPE QUIET -FILTER_NAME gauss_5.0_9x9.conv"
        os.system(sex_cmd)
        cat_tab_r = Table.read(f"{im_name}.cat")
        cat_tab_r.rename_column("X_IMAGE", "xcentroid")
        cat_tab_r.rename_column("Y_IMAGE", "ycentroid")
        _, fwhm, _ = sigma_clipped_stats(cat_tab_r["FWHM_IMAGE"], sigma=3)
        source_coords_r = SkyCoord(ra=cat_tab_r["ALPHA_J2000"], dec=cat_tab_r["DELTA_J2000"], unit="deg", obstime=epoch)
    else:
        sources_r = photometry.detect_sources(red_sci_image, detection_fwhm=10, verbose=False)
        try:
            fwhm = photometry.get_fwhm(red_sci_image, sources_r)
        except RuntimeError:
            fwhm = 10
        cat_tab_r = photometry.apert_photometry(red_sci_image, sources_r, fwhm)
        source_coords_r = SkyCoord.from_pixel(cat_tab_r["xcentroid"], cat_tab_r["xcentroid"], wcs)

In [ ]:
plt.scatter(cat_tab_r["CLASS_STAR"], cat_tab_r["ELONGATION"])
plt.ylim(0.0, 5.0)

In [ ]:
plt.hist(cat_tab_r["CLASS_STAR"], bins=50)

In [ ]:
cat_stars_r = cat_tab_r[cat_tab_r["CLASS_STAR"] > 0.8]
star_coords_r = source_coords_r[cat_tab_r["CLASS_STAR"] > 0.8]

In [ ]:
xm_id, xm_ang_distance, _ = star_coords_r.match_to_catalog_sky(coord_panstarrs, nthneighbor=1)
# print(hdu.header.get('PIXSCALX') * fwhm)
# max_sep = hdu.header.get('PIXSCALX') * fwhm * u.arcsec
max_sep = 2.5 * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches_r = star_coords_r[sep_constraint]
catalog_matches_r = ref_ps[xm_id[sep_constraint]]
coord_catalog_matches_r = coord_panstarrs[xm_id[sep_constraint]]

In [ ]:
# Compute instrumental magnitude
if not use_sextractor:
    exptime = hdu.header.get("EXPTIME")
    ins_mag_r = -2.5 * np.log10(cat_stars_r[sep_constraint]["aper_sum_bkgsub"] / exptime)
    cat_mag_r = ref_ps["rmag"][xm_id[sep_constraint]]

    ins_err_r = ins_mag_r - -2.5 * np.log10((cat_stars_r[sep_constraint]["aper_sum_bkgsub"] + cat_stars_r[sep_constraint]["noise"]) / exptime)
    cat_err_r = ref_ps["e_rmag"][xm_id[sep_constraint]]

    cat_stars_r["ins_mag"] = 99
    cat_stars_r["ins_mag"][sep_constraint] = ins_mag_r
else:
    cat_mag_r = ref_ps["rmag"][xm_id[sep_constraint]]
    cat_err_r = ref_ps["e_rmag"][xm_id[sep_constraint]]
    ins_mag_r = cat_stars_r[sep_constraint]["MAG_BEST"]
    ins_err_r = cat_stars_r[sep_constraint]["MAGERR_BEST"]

sel = ins_mag_r < 99
plt.scatter(ins_mag_r[sel], cat_mag_r[sel])

In [ ]:
# Selection from magnitude range
mag_min, mag_max = 14, 18
cond = (cat_mag_r > mag_min) & (cat_mag_r < mag_max) & (~cat_mag_r.mask) & (~np.isnan(ins_mag_r)) & (ins_mag_r < 99)

# Create two mock arrays for linear regression
X = ins_mag_r[cond].reshape(-1, 1)
y = cat_mag_r[cond].reshape(-1, 1)


# Simple linear regression
linear = linear_model.LinearRegression()
linear.fit(X, y)


# sigma clipping pour choisir le threshold
from scipy import stats

MAD = stats.median_abs_deviation(X - y)
_, _, sig = sigma_clipped_stats(X - y)

print(MAD, sig)

# RANSAC linear regressions
ransac = linear_model.RANSACRegressor(residual_threshold=3 * MAD[0])
# ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)

# Results
print("Photometric calibration:")
print(f"  Linear Slope: {linear.coef_[0][0]:.3f}")
print(f"  Linear ZP   : {linear.intercept_[0]:.3f}\n")
print(f"  RANSAC Slope: {ransac.estimator_.coef_[0][0]:.3f}")
print(f"  RANSAC ZP   : {ransac.estimator_.intercept_[0]:.3f}")

# Plotting regression
# Outliers and Valid points
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Linear regressions (simple and RANSAC)
line_X = np.arange(X.min(), X.max() + 1)[:, np.newaxis]
line_y_simple = linear.predict(line_X)
line_y_ransac = ransac.predict(line_X)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plot data
ax[0].scatter(X[inlier_mask], y[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[0].scatter(X[outlier_mask], y[outlier_mask], color="gray", marker=".", label="Outliers")

# Plot regressions
ax[0].plot(line_X, line_y_simple, color="cornflowerblue", label="Linear regressor")
ax[0].plot(line_X, line_y_ransac, color="navy", label="RANSAC regressor")

# Axes...
ax[0].legend(loc="lower right")
# ax[0].set_ylim([10,18])
ax[0].set_xlabel("Instrument magnitude")
ax[0].set_ylabel("Catalog magnitude")
ax[0].set_aspect("equal")

_, zp_median, zp_sigma = sigma_clipped_stats(y - X, sigma=3)
ax[1].scatter(y[inlier_mask], y[inlier_mask] - X[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[1].scatter(y[outlier_mask], y[outlier_mask] - X[outlier_mask], color="gray", marker=".", label="Outliers")
ax[1].set_xlabel("Catalog magnitude")

ax[1].axhline(zp_median, label="Median")
ax[1].axhline(zp_median + zp_sigma, linestyle="--", label="Standard deviation")
ax[1].axhline(zp_median - zp_sigma, linestyle="--")
print(f"  sigma  ZP   : {zp_sigma:.3f}")

ax[1].set_ylabel("Instrument - Catalog magnitude")
ax[1].legend(loc="best")

In [ ]:
# Compute calibrated mag
cat_tab_r["AB_MAG"] = 99.0

# Positive values
if not use_sextractor:
    positive = np.where(cat_tab_r["aper_sum_bkgsub"] > 0)
    cat_tab_r["AB_MAG"][positive] = ransac.predict((-2.5 * np.log10(cat_tab_r[positive]["aper_sum_bkgsub"] / exptime)).data.reshape(-1, 1)).flatten()
else:
    positive = np.where(cat_tab_r["FLUX_BEST"] > 0)
    cat_tab_r["AB_MAG"][positive] = ransac.predict(cat_tab_r[positive]["MAG_BEST"].data.reshape(-1, 1)).flatten()
cat_tab_r

### I- band

In [ ]:
filt = "i"
exp = 60 if filt == "z" else 30 if filt == "i" else 15  # 10
os.listdir(data_dir)[9] if filt == "z" else os.listdir(data_dir)[8]
red_sci_image = f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_np{combtype}.fits" if manual_stack else f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}_{center_type}.fits"
ref_ps = query_panstarrs(red_sci_image)
coord_panstarrs = SkyCoord(ref_ps["RAJ2000"], ref_ps["DEJ2000"])
im_dir = os.path.abspath(os.path.dirname(red_sci_image))
im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))
with fits.open(red_sci_image) as hdul:
    hdu = hdul[0]
    wcs = WCS(hdu.header)
    epoch = Time(hdu.header.get("MJD-OBS"), format="mjd")
    if use_sextractor:
        sex_cmd = f"source-extractor -c default.sex {red_sci_image} -CATALOG_NAME {im_name}.cat -CATALOG_TYPE FITS_1.0 -VERBOSE_TYPE QUIET -FILTER_NAME gauss_5.0_9x9.conv"
        os.system(sex_cmd)
        cat_tab_i = Table.read(f"{im_name}.cat")
        cat_tab_i.rename_column("X_IMAGE", "xcentroid")
        cat_tab_i.rename_column("Y_IMAGE", "ycentroid")
        _, fwhm, _ = sigma_clipped_stats(cat_tab_i["FWHM_IMAGE"], sigma=3)
        source_coords_i = SkyCoord(ra=cat_tab_i["ALPHA_J2000"], dec=cat_tab_i["DELTA_J2000"], unit="deg", obstime=epoch)
    else:
        sources_i = photometry.detect_sources(red_sci_image, detection_fwhm=10, verbose=False)
        try:
            fwhm = photometry.get_fwhm(red_sci_image, sources_i)
        except RuntimeError:
            fwhm = 10
        cat_tab_i = photometry.apert_photometry(red_sci_image, sources_i, fwhm)
        source_coords_i = SkyCoord.from_pixel(cat_tab_i["xcentroid"], cat_tab_i["xcentroid"], wcs)

In [ ]:
plt.scatter(cat_tab_i["CLASS_STAR"], cat_tab_i["ELONGATION"])
plt.ylim(0.0, 5.0)

In [ ]:
plt.hist(cat_tab_i["CLASS_STAR"], bins=50)

In [ ]:
cat_stars_i = cat_tab_i[cat_tab_i["CLASS_STAR"] > 0.8]
star_coords_i = source_coords_i[cat_tab_i["CLASS_STAR"] > 0.8]
xm_id, xm_ang_distance, _ = star_coords_i.match_to_catalog_sky(coord_panstarrs, nthneighbor=1)
# print(hdu.header.get('PIXSCALX') * fwhm)
# max_sep = hdu.header.get('PIXSCALX') * fwhm * u.arcsec
max_sep = 2.5 * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches_i = star_coords_i[sep_constraint]
catalog_matches_i = ref_ps[xm_id[sep_constraint]]
coord_catalog_matches_i = coord_panstarrs[xm_id[sep_constraint]]
# Compute instrumental magnitude
if not use_sextractor:
    exptime = hdu.header.get("EXPTIME")
    ins_mag_i = -2.5 * np.log10(cat_stars_i[sep_constraint]["aper_sum_bkgsub"] / exptime)
    cat_mag_i = ref_ps["imag"][xm_id[sep_constraint]]

    ins_err_i = ins_mag_i - -2.5 * np.log10((cat_stars_i[sep_constraint]["aper_sum_bkgsub"] + cat_stars_i[sep_constraint]["noise"]) / exptime)
    cat_err_i = ref_ps["e_imag"][xm_id[sep_constraint]]

    cat_stars_i["ins_mag"] = 99
    cat_stars_i["ins_mag"][sep_constraint] = ins_mag_i
else:
    cat_mag_i = ref_ps["imag"][xm_id[sep_constraint]]
    cat_err_i = ref_ps["e_imag"][xm_id[sep_constraint]]
    ins_mag_i = cat_stars_i[sep_constraint]["MAG_BEST"]
    ins_err_i = cat_stars_i[sep_constraint]["MAGERR_BEST"]

sel = ins_mag_i < 99
plt.scatter(ins_mag_i[sel], cat_mag_i[sel])

In [ ]:
# Selection from magnitude range
mag_min, mag_max = 13, 19
cond = (cat_mag_i > mag_min) & (cat_mag_i < mag_max) & (~cat_mag_i.mask) & (~np.isnan(ins_mag_i)) & (ins_mag_i < 99)

# Create two mock arrays for linear regression
X = ins_mag_i[cond].reshape(-1, 1)
y = cat_mag_i[cond].reshape(-1, 1)


# Simple linear regression
linear = linear_model.LinearRegression()
linear.fit(X, y)


# sigma clipping pour choisir le threshold
from scipy import stats

MAD = stats.median_abs_deviation(X - y)
_, _, sig = sigma_clipped_stats(X - y)

print(MAD, sig)

# RANSAC linear regressions
ransac = linear_model.RANSACRegressor(residual_threshold=3 * MAD[0])
# ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)

# Results
print("Photometric calibration:")
print(f"  Linear Slope: {linear.coef_[0][0]:.3f}")
print(f"  Linear ZP   : {linear.intercept_[0]:.3f}\n")
print(f"  RANSAC Slope: {ransac.estimator_.coef_[0][0]:.3f}")
print(f"  RANSAC ZP   : {ransac.estimator_.intercept_[0]:.3f}")

# Plotting regression
# Outliers and Valid points
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Linear regressions (simple and RANSAC)
line_X = np.arange(X.min(), X.max() + 1)[:, np.newaxis]
line_y_simple = linear.predict(line_X)
line_y_ransac = ransac.predict(line_X)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plot data
ax[0].scatter(X[inlier_mask], y[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[0].scatter(X[outlier_mask], y[outlier_mask], color="gray", marker=".", label="Outliers")

# Plot regressions
ax[0].plot(line_X, line_y_simple, color="cornflowerblue", label="Linear regressor")
ax[0].plot(line_X, line_y_ransac, color="navy", label="RANSAC regressor")

# Axes...
ax[0].legend(loc="lower right")
# ax[0].set_ylim([10,18])
ax[0].set_xlabel("Instrument magnitude")
ax[0].set_ylabel("Catalog magnitude")
ax[0].set_aspect("equal")

_, zp_median, zp_sigma = sigma_clipped_stats(y - X, sigma=3)
ax[1].scatter(y[inlier_mask], y[inlier_mask] - X[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[1].scatter(y[outlier_mask], y[outlier_mask] - X[outlier_mask], color="gray", marker=".", label="Outliers")
ax[1].set_xlabel("Catalog magnitude")

ax[1].axhline(zp_median, label="Median")
ax[1].axhline(zp_median + zp_sigma, linestyle="--", label="Standard deviation")
ax[1].axhline(zp_median - zp_sigma, linestyle="--")
print(f"  sigma  ZP   : {zp_sigma:.3f}")

ax[1].set_ylabel("Instrument - Catalog magnitude")
ax[1].legend(loc="best")

In [ ]:
# Compute calibrated mag
cat_tab_i["AB_MAG"] = 99.0

# Positive values
if not use_sextractor:
    positive = np.where(cat_tab_i["aper_sum_bkgsub"] > 0)
    cat_tab_i["AB_MAG"][positive] = ransac.predict((-2.5 * np.log10(cat_tab_i[positive]["aper_sum_bkgsub"] / exptime)).data.reshape(-1, 1)).flatten()
else:
    positive = np.where(cat_tab_i["FLUX_BEST"] > 0)
    cat_tab_i["AB_MAG"][positive] = ransac.predict(cat_tab_i[positive]["MAG_BEST"].data.reshape(-1, 1)).flatten()
cat_tab_i

### Z-mag

In [ ]:
filt = "z"
exp = 30 if filt == "i" else 60 if filt == "z" else 15  # 5
dirchoice = os.listdir(data_dir)[9] if filt == "z" else os.listdir(data_dir)[8]
red_sci_image = f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_np{combtype}.fits" if manual_stack else f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}_{center_type}.fits"
ref_ps = query_panstarrs(red_sci_image)
coord_panstarrs = SkyCoord(ref_ps["RAJ2000"], ref_ps["DEJ2000"])
im_dir = os.path.abspath(os.path.dirname(red_sci_image))
im_name, im_ext = os.path.splitext(os.path.basename(red_sci_image))
with fits.open(red_sci_image) as hdul:
    hdu = hdul[0]
    wcs = WCS(hdu.header)
    epoch = Time(hdu.header.get("MJD-OBS"), format="mjd")
    if use_sextractor:
        sex_cmd = f"source-extractor -c default.sex {red_sci_image} -CATALOG_NAME {im_name}.cat -CATALOG_TYPE FITS_1.0 -VERBOSE_TYPE QUIET -FILTER_NAME gauss_5.0_9x9.conv"
        os.system(sex_cmd)
        cat_tab_z = Table.read(f"{im_name}.cat")
        cat_tab_z.rename_column("X_IMAGE", "xcentroid")
        cat_tab_z.rename_column("Y_IMAGE", "ycentroid")
        _, fwhm, _ = sigma_clipped_stats(cat_tab_z["FWHM_IMAGE"], sigma=3)
        source_coords_z = SkyCoord(ra=cat_tab_z["ALPHA_J2000"], dec=cat_tab_z["DELTA_J2000"], unit="deg", obstime=epoch)
    else:
        sources_z = photometry.detect_sources(red_sci_image, detection_fwhm=10, verbose=False)
        try:
            fwhm = photometry.get_fwhm(red_sci_image, sources_z)
        except RuntimeError:
            fwhm = 10
        cat_tab_z = photometry.apert_photometry(red_sci_image, sources_z, fwhm)
        source_coords_z = SkyCoord.from_pixel(cat_tab_z["xcentroid"], cat_tab_z["xcentroid"], wcs)

In [ ]:
plt.scatter(cat_tab_z["CLASS_STAR"], cat_tab_z["ELONGATION"])
plt.ylim(0.0, 5.0)

In [ ]:
plt.hist(cat_tab_z["CLASS_STAR"], bins=50)

In [ ]:
cat_stars_z = cat_tab_z[cat_tab_z["CLASS_STAR"] > 0.8]
star_coords_z = source_coords_z[cat_tab_z["CLASS_STAR"] > 0.8]
xm_id, xm_ang_distance, _ = star_coords_z.match_to_catalog_sky(coord_panstarrs, nthneighbor=1)
# print(hdu.header.get('PIXSCALX') * fwhm)
# max_sep = hdu.header.get('PIXSCALX') * fwhm * u.arcsec
max_sep = 2.5 * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches_z = star_coords_z[sep_constraint]
catalog_matches_z = ref_ps[xm_id[sep_constraint]]
coord_catalog_matches_z = coord_panstarrs[xm_id[sep_constraint]]
# Compute instrumental magnitude
if not use_sextractor:
    exptime = hdu.header.get("EXPTIME")
    ins_mag_z = -2.5 * np.log10(cat_stars_z[sep_constraint]["aper_sum_bkgsub"] / exptime)
    cat_mag_z = ref_ps["zmag"][xm_id[sep_constraint]]

    ins_err_z = ins_mag_z - -2.5 * np.log10((cat_stars_z[sep_constraint]["aper_sum_bkgsub"] + cat_stars_z[sep_constraint]["noise"]) / exptime)
    cat_err_z = ref_ps["e_zmag"][xm_id[sep_constraint]]

    cat_stars_z["ins_mag"] = 99
    cat_stars_z["ins_mag"][sep_constraint] = ins_mag_z
else:
    cat_mag_z = ref_ps["zmag"][xm_id[sep_constraint]]
    cat_err_z = ref_ps["e_zmag"][xm_id[sep_constraint]]
    ins_mag_z = cat_stars_z[sep_constraint]["MAG_BEST"]
    ins_err_z = cat_stars_z[sep_constraint]["MAGERR_BEST"]

sel = ins_mag_z < 99
plt.scatter(ins_mag_z[sel], cat_mag_z[sel])

In [ ]:
# Selection from magnitude range
mag_min, mag_max = 12, 18
cond = (cat_mag_z > mag_min) & (cat_mag_z < mag_max) & (~cat_mag_z.mask) & (~np.isnan(ins_mag_z)) & (ins_mag_z < 99)

# Create two mock arrays for linear regression
X = ins_mag_z[cond].reshape(-1, 1)
y = cat_mag_z[cond].reshape(-1, 1)


# Simple linear regression
linear = linear_model.LinearRegression()
linear.fit(X, y)


# sigma clipping pour choisir le threshold
from scipy import stats

MAD = stats.median_abs_deviation(X - y)
_, _, sig = sigma_clipped_stats(X - y)

print(MAD, sig)

# RANSAC linear regressions
ransac = linear_model.RANSACRegressor(residual_threshold=3 * MAD[0])
# ransac = linear_model.RANSACRegressor()
ransac.fit(X, y)

# Results
print("Photometric calibration:")
print(f"  Linear Slope: {linear.coef_[0][0]:.3f}")
print(f"  Linear ZP   : {linear.intercept_[0]:.3f}\n")
print(f"  RANSAC Slope: {ransac.estimator_.coef_[0][0]:.3f}")
print(f"  RANSAC ZP   : {ransac.estimator_.intercept_[0]:.3f}")

# Plotting regression
# Outliers and Valid points
inlier_mask = ransac.inlier_mask_
outlier_mask = np.logical_not(inlier_mask)

# Linear regressions (simple and RANSAC)
line_X = np.arange(X.min(), X.max() + 1)[:, np.newaxis]
line_y_simple = linear.predict(line_X)
line_y_ransac = ransac.predict(line_X)

fig, ax = plt.subplots(1, 2, figsize=(15, 5))

# Plot data
ax[0].scatter(X[inlier_mask], y[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[0].scatter(X[outlier_mask], y[outlier_mask], color="gray", marker=".", label="Outliers")

# Plot regressions
ax[0].plot(line_X, line_y_simple, color="cornflowerblue", label="Linear regressor")
ax[0].plot(line_X, line_y_ransac, color="navy", label="RANSAC regressor")

# Axes...
ax[0].legend(loc="lower right")
# ax[0].set_ylim([10,18])
ax[0].set_xlabel("Instrument magnitude")
ax[0].set_ylabel("Catalog magnitude")
ax[0].set_aspect("equal")

_, zp_median, zp_sigma = sigma_clipped_stats(y - X, sigma=3)
ax[1].scatter(y[inlier_mask], y[inlier_mask] - X[inlier_mask], color="yellowgreen", marker=".", label="Inliers")
ax[1].scatter(y[outlier_mask], y[outlier_mask] - X[outlier_mask], color="gray", marker=".", label="Outliers")
ax[1].set_xlabel("Catalog magnitude")

ax[1].axhline(zp_median, label="Median")
ax[1].axhline(zp_median + zp_sigma, linestyle="--", label="Standard deviation")
ax[1].axhline(zp_median - zp_sigma, linestyle="--")
print(f"  sigma  ZP   : {zp_sigma:.3f}")

ax[1].set_ylabel("Instrument - Catalog magnitude")
ax[1].legend(loc="best")

In [ ]:
# Compute calibrated mag
cat_tab_z["AB_MAG"] = 99.0

# Positive values
if not use_sextractor:
    positive = np.where(cat_tab_z["aper_sum_bkgsub"] > 0)
    cat_tab_z["AB_MAG"][positive] = ransac.predict((-2.5 * np.log10(cat_tab_z[positive]["aper_sum_bkgsub"] / exptime)).data.reshape(-1, 1)).flatten()
else:
    positive = np.where(cat_tab_z["FLUX_BEST"] > 0)
    cat_tab_z["AB_MAG"][positive] = ransac.predict(cat_tab_z[positive]["MAG_BEST"].data.reshape(-1, 1)).flatten()
cat_tab_z

## Cross-match G, R, I et Z

In [ ]:
xm_id, xm_ang_distance, _ = source_coords_r.match_to_catalog_sky(source_coords_g, nthneighbor=1)
# print(hdu.header.get('PIXSCALX') * fwhm)
max_sep = 2.5 * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches = source_coords_r[sep_constraint]
catalog_matches = cat_tab_g[xm_id[sep_constraint]]
coord_catalog_matches = source_coords_g[xm_id[sep_constraint]]

In [ ]:
g_cat = catalog_matches
r_cat = cat_tab_r[sep_constraint]

sel = np.logical_and(g_cat["AB_MAG"] < 99.0, r_cat["AB_MAG"] < 99.0)
plt.scatter(g_cat[sel]["AB_MAG"] - r_cat[sel]["AB_MAG"], g_cat[sel]["AB_MAG"], marker=".")

In [ ]:
rg_cat = g_cat.copy()

In [ ]:
rg_cat.columns

In [ ]:
rg_cat.rename_column("FLUX_BEST", "FLUX_BEST_G")
rg_cat.rename_column("FLUXERR_BEST", "FLUXERR_BEST_G")
rg_cat.rename_column("MAG_BEST", "MAG_BEST_G")
rg_cat.rename_column("MAGERR_BEST", "MAGERR_BEST_G")
rg_cat.rename_column("AB_MAG", "AB_MAG_G")

In [ ]:
rg_cat["FLUX_BEST_R"] = r_cat["FLUX_BEST"]
rg_cat["FLUXERR_BEST_R"] = r_cat["FLUXERR_BEST"]
rg_cat["MAG_BEST_R"] = r_cat["MAG_BEST"]
rg_cat["MAGERR_BEST_R"] = r_cat["MAGERR_BEST"]
rg_cat["AB_MAG_R"] = r_cat["AB_MAG"]

In [ ]:
rg_cat

In [ ]:
# joined_coords = SkyCoord(ra=rg_cat["ALPHA_J2000"], dec=rg_cat["DELTA_J2000"], unit="deg", obstime=epoch)
xm_id, xm_ang_distance, _ = source_coords_i.match_to_catalog_sky(coord_catalog_matches, nthneighbor=1)
# print(hdu.header.get('PIXSCALX') * fwhm)
max_sep = 2.5 * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches = source_coords_i[sep_constraint]
catalog_matches = rg_cat[xm_id[sep_constraint]]
coord_catalog_matches = coord_catalog_matches[xm_id[sep_constraint]]

i_cat = cat_tab_i[sep_constraint]
sel = np.logical_and(catalog_matches["AB_MAG_R"] < 99.0, np.logical_and(catalog_matches["AB_MAG_G"] < 99.0, i_cat["AB_MAG"] < 99.0))
plt.scatter(catalog_matches[sel]["AB_MAG_R"] - i_cat[sel]["AB_MAG"], i_cat[sel]["AB_MAG"], marker=".")

rgi_cat = catalog_matches.copy()

rgi_cat["FLUX_BEST_I"] = i_cat["FLUX_BEST"]
rgi_cat["FLUXERR_BEST_I"] = i_cat["FLUXERR_BEST"]
rgi_cat["MAG_BEST_I"] = i_cat["MAG_BEST"]
rgi_cat["MAGERR_BEST_I"] = i_cat["MAGERR_BEST"]
rgi_cat["AB_MAG_I"] = i_cat["AB_MAG"]

rgi_cat

In [ ]:
xm_id, xm_ang_distance, _ = source_coords_z.match_to_catalog_sky(coord_catalog_matches, nthneighbor=1)
# print(hdu.header.get('PIXSCALX') * fwhm)
max_sep = 2.5 * u.arcsec
sep_constraint = xm_ang_distance < max_sep
coord_matches = source_coords_z[sep_constraint]
catalog_matches = rgi_cat[xm_id[sep_constraint]]
coord_catalog_matches = coord_catalog_matches[xm_id[sep_constraint]]

z_cat = cat_tab_z[sep_constraint]
sel = np.logical_and(catalog_matches["AB_MAG_I"] < 99.0, z_cat["AB_MAG"] < 99.0)
plt.scatter(catalog_matches[sel]["AB_MAG_I"] - z_cat[sel]["AB_MAG"], z_cat[sel]["AB_MAG"], marker=".")

rgiz_cat = catalog_matches.copy()

rgiz_cat["FLUX_BEST_Z"] = z_cat["FLUX_BEST"]
rgiz_cat["FLUXERR_BEST_Z"] = z_cat["FLUXERR_BEST"]
rgiz_cat["MAG_BEST_Z"] = z_cat["MAG_BEST"]
rgiz_cat["MAGERR_BEST_Z"] = z_cat["MAGERR_BEST"]
rgiz_cat["AB_MAG_Z"] = z_cat["AB_MAG"]

rgiz_cat

On a ici un mélange d'étoiles et de galaxies. Il faut construire le catalogue de galaxies pour la suite.

In [ ]:
plt.scatter(g_cat["CLASS_STAR"], g_cat["ELONGATION"], marker=".", alpha=0.2)
plt.scatter(r_cat["CLASS_STAR"], r_cat["ELONGATION"], marker="x", alpha=0.2)
plt.scatter(i_cat["CLASS_STAR"], i_cat["ELONGATION"], marker="+", alpha=0.2)
plt.scatter(z_cat["CLASS_STAR"], z_cat["ELONGATION"], marker="s", alpha=0.2)
plt.scatter(rgiz_cat["CLASS_STAR"], rgiz_cat["ELONGATION"], marker="v", alpha=0.4)
plt.xlabel("CLASS_STAR")
plt.ylabel("ELONGATION")
plt.ylim(0.0, 5.0)

In [ ]:
plt.hist(g_cat["ELONGATION"], range=(1, 2), bins=50, alpha=0.2)
plt.hist(r_cat["ELONGATION"], range=(1, 2), bins=50, alpha=0.2)
plt.hist(i_cat["ELONGATION"], range=(1, 2), bins=50, alpha=0.2)
plt.hist(z_cat["ELONGATION"], range=(1, 2), bins=50, alpha=0.2)
plt.hist(rgiz_cat["ELONGATION"], range=(1, 2), bins=50, alpha=0.4)
plt.xlabel("ELONGATION")

In [ ]:
plt.hist(g_cat["CLASS_STAR"], bins=50, alpha=0.2)
plt.hist(r_cat["CLASS_STAR"], bins=50, alpha=0.2)
plt.hist(i_cat["CLASS_STAR"], bins=50, alpha=0.2)
plt.hist(z_cat["CLASS_STAR"], bins=50, alpha=0.2)
plt.hist(rgiz_cat["CLASS_STAR"], bins=50, alpha=0.4)
plt.xlabel("CLASS_STAR")

In [ ]:
gal_sel = np.logical_or(rgiz_cat["CLASS_STAR"] <= 0.6, rgiz_cat["ELONGATION"] > 1.1)
rgiz_gal_cat = rgiz_cat[gal_sel]
sel = np.logical_and(rgiz_gal_cat["AB_MAG_G"] < 99.0, np.logical_and(rgiz_gal_cat["AB_MAG_R"] < 99.0, np.logical_and(rgiz_gal_cat["AB_MAG_I"] < 99.0, rgiz_gal_cat["AB_MAG_Z"] < 99.0)))
rgiz_gal_cat = rgiz_gal_cat[sel]

In [ ]:
rgiz_gal_cat

In [ ]:
plt.scatter(rgiz_gal_cat["AB_MAG_G"] - rgiz_gal_cat["AB_MAG_R"], rgiz_gal_cat["AB_MAG_G"])

In [ ]:
plt.hist(rgiz_gal_cat["AB_MAG_G"] - rgiz_gal_cat["AB_MAG_R"])

In [ ]:
if True:
    rgiz_cat.write(f"UGC9412_GRIZ_CAT_{'np' if manual_stack else 'SW_'}{combtype}.fits", format="fits", overwrite=True)

## Photométrie de la cible

In [ ]:
# Get the target position
target = hdr.get("OBJECT")
print(target)
# if not target_type == "asteroid":
try:
    target_coords = SkyCoord.from_name(target)
except NameResolveError:
    target = " ".join(target.split("-"))  # Quick fix for the case 'UGC-9412' and similar.
    target_coords = SkyCoord.from_name(target)
try:
    target_x, target_y = skycoord_to_pixel(target_coords, wcs=wcs)
except NoConvergence:
    pass
target_coords

In [ ]:
wcs = WCS(hdr)
epoch = Time(hdr.get("MJD-OBS"), format="mjd", scale="utc")
sex_coords = SkyCoord(ra=rgiz_cat["ALPHA_J2000"], dec=rgiz_cat["DELTA_J2000"], unit="deg", obstime=epoch)
# dist_to_target = np.power(cat_tab["xcentroid"]-target_x, 2) + np.power(cat_tab["ycentroid"]-target_y, 2)
dist_to_target = sex_coords.separation(target_coords)
id_target = np.nanargmin(dist_to_target)
sex_target_table = rgiz_cat[id_target]
sex_target_table

In [ ]:
from astropy.nddata import Cutout2D
from astropy.visualization.wcsaxes import Quadrangle
from astropy.coordinates import Angle
from regions import PixCoord, EllipsePixelRegion

filt = "r"  # , 'r', 'i', 'z']:
exp = 30 if filt == "i" else 60 if filt == "z" else 15  # 5
dirchoice = os.listdir(data_dir)[9] if filt == "z" else os.listdir(data_dir)[8]
for manual_stack in [True, False]:
    for combtype in ["AVERAGE", "MEDIAN", "SUM"]:
        rgiz_cat = Table.read(f"UGC9412_GRIZ_CAT_{'np' if manual_stack else 'SW_'}{combtype}.fits", format="fits")
        gal_sel = np.logical_or(rgiz_cat["CLASS_STAR"] <= 0.6, rgiz_cat["ELONGATION"] > 1.1)
        rgiz_gal_cat = rgiz_cat[gal_sel]
        sel = np.logical_and(rgiz_gal_cat["AB_MAG_G"] < 99.0, np.logical_and(rgiz_gal_cat["AB_MAG_R"] < 99.0, np.logical_and(rgiz_gal_cat["AB_MAG_I"] < 99.0, rgiz_gal_cat["AB_MAG_Z"] < 99.0)))
        rgiz_gal_cat = rgiz_gal_cat[sel]

        red_sci_image = f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_np{combtype}.fits" if manual_stack else f"coadd_{dirchoice}_SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}_{center_type}.fits"
        lab = f"SDSS{filt}p_{exp:04d}s_RED_np{combtype}" if manual_stack else f"SDSS{filt}p_{exp:04d}s_RED_SW_{combtype}"
        with fits.open(red_sci_image) as stack_gp:
            hdr = stack_gp[0].header
            data = stack_gp[0].data
        wcs = WCS(hdr)

        epoch = Time(hdr.get("MJD-OBS"), format="mjd", scale="utc")
        sex_coords = SkyCoord(ra=rgiz_cat["ALPHA_J2000"], dec=rgiz_cat["DELTA_J2000"], unit="deg", obstime=epoch)
        # dist_to_target = np.power(cat_tab["xcentroid"]-target_x, 2) + np.power(cat_tab["ycentroid"]-target_y, 2)
        dist_to_target = sex_coords.separation(target_coords)
        id_target = np.nanargmin(dist_to_target)
        sex_target_table = rgiz_cat[id_target]

        mean, med, sigma = sigma_clipped_stats(data, sigma=3)

        f = plt.figure(figsize=(10, 5))

        ax0 = f.add_subplot(1, 2, 1, projection=wcs)
        ax0.imshow(data, cmap="gray", vmin=med - 3 * sigma, vmax=med + 17 * sigma)
        # plt.colorbar()
        ax0.scatter(rgiz_cat["xcentroid"], rgiz_cat["ycentroid"], color="y", alpha=0.2, transform=ax0.get_transform("pixel"))
        ax0.scatter(rgiz_gal_cat["xcentroid"], rgiz_gal_cat["ycentroid"], color="r", alpha=0.3, transform=ax0.get_transform("pixel"))
        ax0.scatter(sex_target_table["xcentroid"], sex_target_table["ycentroid"], color="g", alpha=0.5, transform=ax0.get_transform("pixel"))
        ax0.set_title(lab)

        size = 45 * u.arcsec
        orig = (sex_target_table["ALPHA_J2000"] * u.deg - size / 2, sex_target_table["DELTA_J2000"] * u.deg - size / 2)
        center_sky = SkyCoord(ra=(orig[0] + size / 2).to(u.deg), dec=(orig[1] + size / 2).to(u.deg), unit=u.deg)
        center_pix = center_sky.to_pixel(wcs=wcs)
        r = Quadrangle(orig, size, size, edgecolor="yellow", facecolor="none", transform=ax0.get_transform("icrs"))
        ax0.add_patch(r)
        ax0.coords.grid(color="white", linestyle="solid", alpha=0.5)

        zoom = Cutout2D(data, position=(sex_target_table["xcentroid"], sex_target_table["ycentroid"]), size=size, wcs=wcs)
        ax1 = f.add_subplot(1, 2, 2, projection=zoom.wcs)
        ax1.imshow(zoom.data, vmin=med - 5 * sigma, vmax=med + 77 * sigma)
        ax1.coords.grid(color="yellow", linestyle="solid", alpha=0.5)

        center = PixCoord.from_sky(target_coords, wcs=zoom.wcs)
        reg = EllipsePixelRegion(center, width=sex_target_table["A_IMAGE"], height=sex_target_table["B_IMAGE"], angle=Angle(sex_target_table["THETA_IMAGE"], "deg"))
        patch = reg.plot(ax=ax1, facecolor="none", edgecolor="red", lw=1, label="Source")

        z_target = 0.031328
        context = 0
        photom_table = np.array([(sex_target_table[f"AB_MAG_{F}"], sex_target_table[f"MAGERR_BEST_{F}"]) for F in ("G", "R", "I", "Z")])
        photom_table = photom_table.flatten()
        lp_input = np.array([[sex_target_table["NUMBER"], *photom_table, context, z_target]])
        fmt_list = ("%.1i", "%.6g", "%.6g", "%.6g", "%.6g", "%.6g", "%.6g", "%.6g", "%.6g", "%.1i", "%.6f")
        np.savetxt(f"UGC9412_GRIZ_CAT_{'np' if manual_stack else 'SW_'}{combtype}.in", lp_input, fmt=fmt_list, delimiter="  ")
# f.add_axes(ax1)

In [ ]:
dist_to_target[id_target]

In [ ]:
sex_target_table["CLASS_STAR"]

In [ ]:
sex_target_table["ELONGATION"]

Alors que la cible est bonne... on est bien en présence d'une galaxie !

In [ ]:
z_target = 0.031328
context = 0

In [ ]:
photom_table = np.array([(sex_target_table[f"AB_MAG_{F}"], sex_target_table[f"MAGERR_BEST_{F}"]) for F in ("G", "R", "I", "Z")])
photom_table = photom_table.flatten()
photom_table

In [ ]:
lp_input = np.array([[sex_target_table["NUMBER"], *photom_table, context, z_target]])
lp_input.shape

In [ ]:
fmt_list = ("%.1i", "%.6g", "%.6g", "%.6g", "%.6g", "%.6g", "%.6g", "%.6g", "%.6g", "%.1i", "%.6f")
print(len(fmt_list))
np.savetxt(f"UGC9412_GRIZ_CAT_{'np' if manual_stack else 'SW_'}{combtype}.in", lp_input, fmt=fmt_list, delimiter="  ")

In [ ]:
check_cat_arr = np.loadtxt(f"UGC9412_GRIZ_CAT_{'np' if manual_stack else 'SW_'}{combtype}.in")

In [ ]:
check_cat_arr